#### Loading EEG data from .csv files
Before loading the .csv files, the appropriate files should be placed in their respective directories.

For example:

    Directory Name: "solo_subject_1"
    FileNames: "group_01_sujet_01.csv", "group_02_sujet_01.csv", ... , "group_19_sujet_01.csv"
    
    Directory Name: "solo_subject_2"
    FileNames: "group_01_sujet_02.csv", "group_02_sujet_02.csv", ... , "group_19_sujet_02.csv"
    
    Directory Name: "dual"
    FileNames: "group_01.csv", "group_02.csv", "group_03.csv", ... , "group_19.csv"

#### Import libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne.io import RawArray
from mne import Epochs, find_events

#### Load csv -> df -> raw -> filter -> epoch

In [2]:


class Load_Data():
    def __init__(self, header, tmax, tmin, url = None):
        if url == None:
            self.url = os.getcwd()
        else:
            self.url = url
        self.header = header
        self.tmin = tmin
        self.tmax = tmax
        
    
    def load_csv(self, file_name, subject):
        file_url = os.path.join(self.url, file_name)
        df = pd.read_csv(file_url, delimiter = ',', dtype = 'float64', header = None)
        df = df.drop([0], axis = 1)
        
        if subject == 1:
            df = df = pd.concat((df.iloc[:, :32], df.iloc[:, -1:]), axis = 1)
        elif subject == 2:
            df = df = pd.concat((df.iloc[:, 32:-2], df.iloc[:, -1:]), axis = 1)
        else:
            raise ValueError("Subject is not correct. Please select subject 1 or 2!")
        
        df.columns = self.header
        return df
        
    def df_to_raw(self, df):
        df.T[:-1] *= 1e-6
        sfreq = 512
        ch_types = ['eeg'] * 32 + ['stim']
        montage = mne.channels.make_standard_montage('standard_1020')
        info = mne.create_info(ch_names=self.header, ch_types = ch_types, sfreq = sfreq)
        raw = mne.io.RawArray(df.T, info)
        raw.set_montage(montage)
        return raw
        
    def filtering(self, raw):
        raw.notch_filter(np.arange(50, 256, 50))
#         print('===== notch filtered =====')
#         raw.plot_psd()
        
        raw.filter(1, 20, method = 'iir')
#         print('===== band pass filtered =====')
#         raw.plot_psd(fmax = 60)
        return raw

# for Machine Learning Model, noise filtering and ICA may be required in between these steps

    def epoching (self, raw):
        eeg_channels = mne.pick_types(raw.info, eeg = True)
        picks = eeg_channels
        events = find_events(raw)
        reject_criteria = dict(eeg=100e-6)
        event_id = {'Non-Target' : 1, "Target" : 2}
        epochs = Epochs(raw, events = events, event_id = event_id, tmin = self.tmin, tmax = self.tmax,
                       picks = picks, preload = True)
        print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)
        return epochs
        
    def make_epoch (self, file_name, subject):
        df = self.load_csv(file_name, subject)
        raw = self.df_to_raw(df)
        raw = self.filtering(raw)
        epochs = self.epoching(raw)
        return epochs

#### Change the variables
- Place the input .csv including folder at the same level directory as this current file directory.
- Change the folder name below

In [7]:
# to change subjects and player_type to use & folder name
params = {
            "url" : os.path.join(os.getcwd(), 'solo_subject_2'),
            "header" : ['Fp1', 'Fp2', 'AFz', 'F7', 'F3', 'F4', 'F8','FC5', 'FC1', 'FC2', 'FC6',
                  'T7', 'C3', 'Cz','C4', 'T8', 'CP5', 'CP1', 'CP2', 'CP6', 'P7','P3',
                  'Pz', 'P4', 'P8', 'PO7', 'O1', 'Oz','O2', 'PO8', 'PO9', 'PO10','Target'],
            "tmin" : -0.1,
            "tmax" : 0.7
         }


player_type = 'solo' ### Notice if player_type is dual, then subjects must be a list of [1, 2]
subjects = [2]
file_names = os.listdir(params['url'])

#### Loop subjects, loop files and save to local as .npy file
- epoch.save() to save the epoches as _epo.fif file
- can also change the directory to save the _epo.fif file

In [8]:
for subject in subjects:
    load_data = Load_Data(**params)
    for i, file_name in enumerate(file_names):
        epoch = load_data.make_epoch(file_name, subject)
# Change the folder directory as required!
        epoch.save('./data/'+'g'+str(i+1)+player_type+str(subject)+'_epo.fif')
#         break

Creating RawArray with float64 data, n_channels=33, n_times=110528
    Range : 0 ... 110527 =      0.000 ...   215.873 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 20.00 Hz: -6.02, -6.02 dB

252 events found
Event IDs: [1 2]
Not setting metadata
Not setting metadata
252 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correct


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 20.00 Hz: -6.02, -6.02 dB

300 events found
Event IDs: [1 2]
Not setting metadata
Not setting metadata
300 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 300 events and 410 original time points ...
0 bad epochs dropped
sample drop %:  0.0
Creating Ra

- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 20.00 Hz: -6.02, -6.02 dB

216 events found
Event IDs: [1 2]
Not setting metadata
Not setting metadata
216 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 216 events and 410 original time points ...
0 bad epochs dropped
sample drop %:  0.0
Creating RawArray with float64 data, n_channels=33, n_times=119168
    Range : 0 ... 119167 =      0.000 ...   232.748 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, no